In [1]:
import pandas as pd
import fuzzywuzzy as fuzz
from thefuzz import fuzz
from thefuzz import process

/Users/azib/opt/anaconda3/envs/irs/lib/python3.7/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Small DataFrame of IRS990PF

In [2]:
pf_names = pd.read_csv('To_Azib/pf_names_v2.csv') # need to add city state to fip conversion (use most common fip)

## Bonica Data

In [3]:
dime = pd.read_csv('dime_contributors_1979_2014.csv',encoding="ISO-8859-1")

/Users/azib/opt/anaconda3/envs/irs/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
dime.columns

Index(['bonica.cid', 'contributor.type', 'num.records', 'num.distinct',
       'most.recent.contributor.name', 'most.recent.contributor.address',
       'most.recent.contributor.city', 'most.recent.contributor.zipcode',
       'most.recent.contributor.state', 'most.recent.contributor.occupation',
       'most.recent.contributor.employer', 'most.recent.transaction.id',
       'contributor.gender', 'is_corp', 'contributor.cfscore', 'is.projected',
       'first_cycle_active', 'last_cycle_active', 'amount_1980', 'amount_1982',
       'amount_1984', 'amount_1986', 'amount_1988', 'amount_1990',
       'amount_1992', 'amount_1994', 'amount_1996', 'amount_1998',
       'amount_2000', 'amount_2002', 'amount_2004', 'amount_2006',
       'amount_2008', 'amount_2010', 'amount_2012', 'amount_2014'],
      dtype='object')

## ZIPS and FIPS code Data

In [5]:
codes = pd.read_excel('ZIP_COUNTY_122021.xlsx')
codes['zip'] = codes['zip'].astype(str).str.zfill(5)
codes.head()

,zip,county,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00683,72125,SAN GERMAN,PR,0.953036,0.996656,0.980769,0.956300
1,00683,72079,SAN GERMAN,PR,0.000949,0.000000,0.000000,0.000873
2,00683,72023,SAN GERMAN,PR,0.000791,0.001115,0.000000,0.000800
3,00683,72097,SAN GERMAN,PR,0.000158,0.000000,0.000000,0.000145
4,00683,72121,SAN GERMAN,PR,0.045066,0.002230,0.019231,0.041882


In [6]:
codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54260 entries, 0 to 54259
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   zip                  54260 non-null  object 
 1   county               54260 non-null  int64  
 2   usps_zip_pref_city   54260 non-null  object 
 3   usps_zip_pref_state  54260 non-null  object 
 4   res_ratio            54260 non-null  float64
 5   bus_ratio            54260 non-null  float64
 6   oth_ratio            54260 non-null  float64
 7   tot_ratio            54260 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 3.3+ MB


In [7]:
codes.loc[codes['tot_ratio'].idxmax()]

zip                       00923
county                    72127
usps_zip_pref_city     SAN JUAN
usps_zip_pref_state          PR
res_ratio                   1.0
bus_ratio                   1.0
oth_ratio                   1.0
tot_ratio                   1.0
Name: 5, dtype: object

In [8]:
test = codes[['zip','county','usps_zip_pref_city','usps_zip_pref_state']].loc[codes.sort_values(by=['tot_ratio'],ascending=False).tot_ratio >= 0.90]
test['zip'] = test['zip'].astype(str).str.zfill(5)


In [9]:
pf_names.gm_city = pf_names.gm_city.str.upper()
pf_names.first_name = pf_names.first_name.str.upper()
pf_names.last_name = pf_names.last_name.str.upper()
pf_names.name = pf_names.name.str.upper()

In [10]:
pf_names_updated = pd.merge(left=pf_names,right=test,right_on=['usps_zip_pref_city','usps_zip_pref_state'],left_on=['gm_city','gm_state'])
pf_names_updated.drop(['usps_zip_pref_city','usps_zip_pref_state'],axis=1,inplace=True)

In [11]:
pf_names_updated

,BoardExCompanyID,CompanyName,GrantmakerName,ein,gm_city,gm_state,taxyear,name,name_json,name_title,first_name,middle_initials,last_name,duplicated,name_non_alphanum,zip,county
0,480061.0,Masimo Corporation,"The Masimo Foundation for Ethics, Innovation a...",10956020,IRVINE,CA,2017,DAVID VAN RAMSHORST,0,NaN,DAVID,NaN,VAN RAMSHORST,0,0,92612,6059
1,480061.0,Masimo Corporation,"The Masimo Foundation for Ethics, Innovation a...",10956020,IRVINE,CA,2017,DAVID VAN RAMSHORST,0,NaN,DAVID,NaN,VAN RAMSHORST,0,0,92603,6059
2,480061.0,Masimo Corporation,"The Masimo Foundation for Ethics, Innovation a...",10956020,IRVINE,CA,2017,DAVID VAN RAMSHORST,0,NaN,DAVID,NaN,VAN RAMSHORST,0,0,92606,6059
3,480061.0,Masimo Corporation,"The Masimo Foundation for Ethics, Innovation a...",10956020,IRVINE,CA,2017,DAVID VAN RAMSHORST,0,NaN,DAVID,NaN,VAN RAMSHORST,0,0,92623,6059
4,480061.0,Masimo Corporation,"The Masimo Foundation for Ethics, Innovation a...",10956020,IRVINE,CA,2017,DAVID VAN RAMSHORST,0,NaN,DAVID,NaN,VAN RAMSHORST,0,0,92617,6059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307645,22521.0,Northrop Grumman Corporation,The Northrop Grumman Foundation,956095343,FALLS CHURCH,VA,2018,SHEILA CHESTON,0,NaN,SHEILA,NaN,CHESTON,0,0,22041,51059
307646,22521.0,Northrop Grumman Corporation,The Northrop Grumman Foundation,956095343,FALLS CHURCH,VA,2018,TIFFANY KING,0,NaN,TIFFANY,NaN,KING,0,0,22043,51059
307647,22521.0,Northrop Grumman Corporation,The Northrop Grumman Foundation,956095343,FALLS CHURCH,VA,2018,TIFFANY KING,0,NaN,TIFFANY,NaN,KING,0,0,22042,51059
307648,22521.0,Northrop Grumman Corporation,The Northrop Grumman Foundation,956095343,FALLS CHURCH,VA,2018,TIFFANY KING,0,NaN,TIFFANY,NaN,KING,0,0,22040,51610


## Adding FIPS codes to dime database

In [12]:
dime.columns

Index(['bonica.cid', 'contributor.type', 'num.records', 'num.distinct',
       'most.recent.contributor.name', 'most.recent.contributor.address',
       'most.recent.contributor.city', 'most.recent.contributor.zipcode',
       'most.recent.contributor.state', 'most.recent.contributor.occupation',
       'most.recent.contributor.employer', 'most.recent.transaction.id',
       'contributor.gender', 'is_corp', 'contributor.cfscore', 'is.projected',
       'first_cycle_active', 'last_cycle_active', 'amount_1980', 'amount_1982',
       'amount_1984', 'amount_1986', 'amount_1988', 'amount_1990',
       'amount_1992', 'amount_1994', 'amount_1996', 'amount_1998',
       'amount_2000', 'amount_2002', 'amount_2004', 'amount_2006',
       'amount_2008', 'amount_2010', 'amount_2012', 'amount_2014'],
      dtype='object')

In [13]:
dime['most.recent.contributor.zipcode']

0                   0
1                   0
2               62541
3           752014757
4           752044757
              ...    
16356369        33486
16356370        48326
16356371        29201
16356372        85260
16356373        77571
Name: most.recent.contributor.zipcode, Length: 16356374, dtype: int64

In [14]:
dime['most.recent.contributor.zipcode'] = dime['most.recent.contributor.zipcode'].astype(str).str.zfill(5)
dime['most.recent.contributor.city'] = dime['most.recent.contributor.city'].str.upper()
dime['zipcode.5digits'] = dime['most.recent.contributor.zipcode'].str[:5]
dime.head(10)

,bonica.cid,contributor.type,num.records,num.distinct,most.recent.contributor.name,most.recent.contributor.address,most.recent.contributor.city,most.recent.contributor.zipcode,most.recent.contributor.state,most.recent.contributor.occupation,...,amount_1998,amount_2000,amount_2002,amount_2004,amount_2006,amount_2008,amount_2010,amount_2012,amount_2014,zipcode.5digits
0,1.000000e+00,C,114,62,UFWA COPE COMMITTEE,1910 air ln dr,NASHVILLE,00000,TN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000
1,1.000320e+05,C,10,13,COMMITTEE TO RE ELECT CHARLES E. BENNETT,100 riverside ave,JACKSONVILLE,00000,FL,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000
2,1.001611e+09,I,3,3,"donovan, g scott",92 eager rd,NORTH FRANKLIN,62541,CT,NaN,...,0.0,0.0,0.0,0.0,250.0,300.0,0.0,100.0,0.0,62541
3,1.001617e+09,I,25,3,"harrison, joe",NaN,DALLAS,752014757,tx,attorney,...,0.0,0.0,473.0,6184.0,4915.0,2799.0,6273.0,2426.0,2058.0,75201
4,1.001617e+09,I,57,18,"harrison, douglas",NaN,DALLAS,752044757,tx,attorney,...,0.0,0.0,1280.0,8115.0,6953.0,2481.0,8761.0,2195.0,3019.0,75204
5,1.001617e+09,I,3,2,"harrison, mark",po box 695,PORTLAND,37148,TN,vp loan review officer,...,0.0,0.0,0.0,0.0,0.0,0.0,320.0,100.0,0.0,37148
6,1.001633e+09,I,3,2,"fletcher, dr william s",NaN,PORTLAND,97225,OR,retired,...,0.0,0.0,0.0,0.0,0.0,200.0,250.0,0.0,0.0,97225
7,1.001634e+09,I,5,2,"douglas, stephen",NaN,HOUSTON,77002,tx,attorney,...,0.0,0.0,0.0,361.0,350.0,0.0,0.0,0.0,0.0,77002
8,1.002032e+07,I,4,2,"sanchez, dennis",NaN,LAKEWOOD,90712,CA,physician,...,0.0,0.0,0.0,0.0,0.0,0.0,450.0,250.0,0.0,90712
9,1.002612e+09,I,6,6,"curtis, barry",2510 ivanridge cir,GARLAND,750444688,TX,entrepreneur,...,0.0,0.0,0.0,0.0,0.0,1000.0,620.0,100.0,2650.0,75044


In [15]:
new_data = pd.merge(left=dime,right=test,right_on=['zip','usps_zip_pref_city','usps_zip_pref_state'],left_on=['zipcode.5digits','most.recent.contributor.city','most.recent.contributor.state']).drop(['usps_zip_pref_city','usps_zip_pref_state','zip'],axis=1)


In [16]:
new_data.head()

,bonica.cid,contributor.type,num.records,num.distinct,most.recent.contributor.name,most.recent.contributor.address,most.recent.contributor.city,most.recent.contributor.zipcode,most.recent.contributor.state,most.recent.contributor.occupation,...,amount_2000,amount_2002,amount_2004,amount_2006,amount_2008,amount_2010,amount_2012,amount_2014,zipcode.5digits,county
0,1.001617e+09,I,3,2,"harrison, mark",po box 695,PORTLAND,37148,TN,vp loan review officer,...,0.0,0.0,0.0,0.0,0.0,320.0,100.0,0.0,37148,47165
1,2.312652e+09,I,5,4,"tomlinson, troy v",793 n russell st,PORTLAND,371482014,TN,executive,...,0.0,0.0,0.0,500.0,0.0,500.0,500.0,500.0,37148,47165
2,2.374686e+09,I,11,2,"green, e a",108 high st,PORTLAND,37148,TN,retired,...,0.0,70.0,110.0,0.0,0.0,0.0,0.0,0.0,37148,47165
3,2.397701e+09,I,9,5,"hall, ragan",780 n leath rd,PORTLAND,37148,TN,realty and auction,...,0.0,0.0,1500.0,1200.0,675.0,0.0,1000.0,0.0,37148,47165
4,2.419726e+09,I,10,7,"barker, gary",826 s brdway,PORTLAND,37148,TN,NaN,...,0.0,0.0,250.0,275.0,600.0,375.0,400.0,0.0,37148,47165


### Split names in dime to last name and first name

In [17]:
new_data['most.recent.contributor.name'] = new_data['most.recent.contributor.name'].str.upper()
new_data[['last name', 'first name']] = new_data['most.recent.contributor.name'].str.split(',', 1, expand=True)
#new_data['last name'] = new_data['last name'].str.upper()
#new_data['first name'] = new_data['first name'].str.upper()

In [18]:
new_data.head()

,bonica.cid,contributor.type,num.records,num.distinct,most.recent.contributor.name,most.recent.contributor.address,most.recent.contributor.city,most.recent.contributor.zipcode,most.recent.contributor.state,most.recent.contributor.occupation,...,amount_2004,amount_2006,amount_2008,amount_2010,amount_2012,amount_2014,zipcode.5digits,county,last name,first name
0,1.001617e+09,I,3,2,"HARRISON, MARK",po box 695,PORTLAND,37148,TN,vp loan review officer,...,0.0,0.0,0.0,320.0,100.0,0.0,37148,47165,HARRISON,MARK
1,2.312652e+09,I,5,4,"TOMLINSON, TROY V",793 n russell st,PORTLAND,371482014,TN,executive,...,0.0,500.0,0.0,500.0,500.0,500.0,37148,47165,TOMLINSON,TROY V
2,2.374686e+09,I,11,2,"GREEN, E A",108 high st,PORTLAND,37148,TN,retired,...,110.0,0.0,0.0,0.0,0.0,0.0,37148,47165,GREEN,E A
3,2.397701e+09,I,9,5,"HALL, RAGAN",780 n leath rd,PORTLAND,37148,TN,realty and auction,...,1500.0,1200.0,675.0,0.0,1000.0,0.0,37148,47165,HALL,RAGAN
4,2.419726e+09,I,10,7,"BARKER, GARY",826 s brdway,PORTLAND,37148,TN,NaN,...,250.0,275.0,600.0,375.0,400.0,0.0,37148,47165,BARKER,GARY


In [19]:
new_data.columns

Index(['bonica.cid', 'contributor.type', 'num.records', 'num.distinct',
       'most.recent.contributor.name', 'most.recent.contributor.address',
       'most.recent.contributor.city', 'most.recent.contributor.zipcode',
       'most.recent.contributor.state', 'most.recent.contributor.occupation',
       'most.recent.contributor.employer', 'most.recent.transaction.id',
       'contributor.gender', 'is_corp', 'contributor.cfscore', 'is.projected',
       'first_cycle_active', 'last_cycle_active', 'amount_1980', 'amount_1982',
       'amount_1984', 'amount_1986', 'amount_1988', 'amount_1990',
       'amount_1992', 'amount_1994', 'amount_1996', 'amount_1998',
       'amount_2000', 'amount_2002', 'amount_2004', 'amount_2006',
       'amount_2008', 'amount_2010', 'amount_2012', 'amount_2014',
       'zipcode.5digits', 'county', 'last name', 'first name'],
      dtype='object')

In [20]:
new_data.head(2)

,bonica.cid,contributor.type,num.records,num.distinct,most.recent.contributor.name,most.recent.contributor.address,most.recent.contributor.city,most.recent.contributor.zipcode,most.recent.contributor.state,most.recent.contributor.occupation,...,amount_2004,amount_2006,amount_2008,amount_2010,amount_2012,amount_2014,zipcode.5digits,county,last name,first name
0,1.001617e+09,I,3,2,"HARRISON, MARK",po box 695,PORTLAND,37148,TN,vp loan review officer,...,0.0,0.0,0.0,320.0,100.0,0.0,37148,47165,HARRISON,MARK
1,2.312652e+09,I,5,4,"TOMLINSON, TROY V",793 n russell st,PORTLAND,371482014,TN,executive,...,0.0,500.0,0.0,500.0,500.0,500.0,37148,47165,TOMLINSON,TROY V


- Match 

---

In [21]:
test = new_data[100:201]
test['fname_ratio'] = test.apply(lambda x: fuzz.ratio(test['first name'].values[0], pf_names['first_name'].values[0]),axis=1)

/Users/azib/opt/anaconda3/envs/irs/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
test

,bonica.cid,contributor.type,num.records,num.distinct,most.recent.contributor.name,most.recent.contributor.address,most.recent.contributor.city,most.recent.contributor.zipcode,most.recent.contributor.state,most.recent.contributor.occupation,...,amount_2006,amount_2008,amount_2010,amount_2012,amount_2014,zipcode.5digits,county,last name,first name,fname_ratio
100,2.454751e+09,I,1,1,"KELLEY, SCOTT",154 oakhill dr,PORTLAND,37148,TN,NaN,...,0.0,0.0,0.0,0.0,0.0,37148,47165,KELLEY,SCOTT,0
101,2.454756e+09,I,1,1,"BEST, KEN",NaN,PORTLAND,37148,TN,retired,...,0.0,0.0,250.0,0.0,0.0,37148,47165,BEST,KEN,0
102,2.458762e+09,I,2,1,"BRILEY, WAYNE",NaN,PORTLAND,37148,TN,NaN,...,0.0,0.0,0.0,0.0,0.0,37148,47165,BRILEY,WAYNE,0
103,2.458762e+09,I,1,1,"BRILEY, MELVIN",500 n russell st,PORTLAND,37148,TN,NaN,...,0.0,0.0,0.0,0.0,0.0,37148,47165,BRILEY,MELVIN,0
104,2.458762e+09,I,1,1,"BRILEY, ANN",NaN,PORTLAND,37148,TN,NaN,...,0.0,0.0,0.0,0.0,0.0,37148,47165,BRILEY,ANN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,3.105042e+09,I,1,1,"WILLIAMS, CHARLES",345 s leath rd,PORTLAND,37148,TN,attorney,...,0.0,0.0,0.0,0.0,0.0,37148,47165,WILLIAMS,CHARLES,0
197,3.111071e+09,I,1,1,"DOUGLAS, KEITH",1009 goldfinch trail,PORTLAND,37148,TN,ems director,...,0.0,0.0,35.0,0.0,0.0,37148,47165,DOUGLAS,KEITH,0
198,3.113936e+10,I,1,1,"SUDDARTH, ANTHONY",425 s martin rd,PORTLAND,37148,TN,NaN,...,0.0,0.0,50.0,0.0,0.0,37148,47165,SUDDARTH,ANTHONY,0
199,3.135105e+09,I,1,1,"ODEN, JACK",103 mason st,PORTLAND,37148,TN,NaN,...,0.0,0.0,0.0,0.0,0.0,37148,47165,ODEN,JACK,0


In [23]:
new_data['zipcode.5digits'].nunique()

33772

In [24]:
from tqdm.auto import tqdm

In [ ]:
for i in tqdm(new_data.county.unique()):
    a = pd.DataFrame(new_data[new_data.county == i])
    a.to_csv(f'dataframes_fips/data_fips_{i}.csv')
for j in tqdm(new_data['zipcode.5digits'].unique()):
    b = pd.DataFrame(new_data[new_data['zipcode.5digits'] == j])
    b.to_csv(f'dataframes_zips/data_zips_{j}.csv')


  0%|          | 0/3192 [00:00<?, ?it/s]

  0%|          | 0/33772 [00:00<?, ?it/s]

---

In [62]:
data = pd.read_csv('dataframes_fips/data_fips_1001.csv',index_col=0)